<a href="https://colab.research.google.com/github/h-rathee30/Link-Prediction-/blob/master/Implementedpaper2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving dolphins.gml to dolphins.gml


In [ ]:
#if the data is in gml file format
G = nx.read_gml("dolphins.gml")
G = nx.convert_node_labels_to_integers(G, first_label=1, ordering='default')
data = list(G.edges)
data = pd.DataFrame(data)
data

In [ ]:
#if the data is in a .txt file
from sklearn.model_selection import train_test_split
data = pd.read_csv('b1.in',delim_whitespace=True, names=('SRC', 'TGT') )
data

In [ ]:
#data editiing with respect to the data uploaded 
data.drop(['IGNORE'], axis = 1, inplace = True)
data = data.iloc[1:50001]
data.reset_index(drop = True, inplace = True)
data = data.astype(int)

In [ ]:
No_of_nodes = data.to_numpy()
No_of_nodes = np.unique(No_of_nodes)
nodes_input = np.sort(No_of_nodes)

In [ ]:
def Pair_Of_Nodes(g):
    nodes = list(g.nodes) # list is used bec. graph nodes provides nodeView 
    pair_of_nodes = []
    for i in range(0, len(nodes)):
        for j in range(i+1, len(nodes)) :
            pair = [nodes[i], nodes[j]]
            pair_of_nodes.append(pair)
    return pair_of_nodes


def Common_NeighboursList(g, all_Pairs):
    commonNeighoursList = {}
    for node1, node2 in all_Pairs :
        commonNeighoursList[str(node1)+" "+str(node2)] = len(list(nx.common_neighbors(g,node1,node2)))
    return  commonNeighoursList 


#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b,g):
    p=-1
    try:
        if g.has_edge(a,b):
            g.remove_edge(a,b)
            p= len(nx.shortest_path(g,source=a,target=b))-1
            g.add_edge(a,b)
        else:
            p= len(nx.shortest_path(g,source=a,target=b))-1
        return p
    except:
        return -1


def Closeness_Centrality(u, v, g):
    short_dist = compute_shortest_path_length(u,v,g)
    no_of_nodes_in_connected_part = len(No_of_nodes)
    if short_dist == -1:
        return 0
    else:
        return (no_of_nodes_in_connected_part/short_dist)


def Common_Neighbours(g, pair_of_nodes):
    common_neighbours = []
    for node1, node2 in pair_of_nodes:
        length = len(list((nx.common_neighbors(g,node1,node2))))
        temp = [node1, node2, length ]
        common_neighbours.append(temp) 
    return common_neighbours


def getDictionaryFromList(input_list) :
    dic = {}
    for ele in input_list :
        dic[str(ele[0])+" "+str(ele[1])] = ele[2]
    return dic

In [ ]:
independent_variable = {}
dependent_variable = {}
train, test = train_test_split(data, test_size = 0.2)
train = train.values.tolist()
test = test.values.tolist()

tra = []
tst = []
testSet = set()
for dt in range(0,len(train)) :
    node1 = train[dt][0] if train[dt][0] <= train[dt][1] else train[dt][1]  
    node2 = train[dt][0] if train[dt][0] >= train[dt][1] else train[dt][1]
    if node1 == node2 : continue
    else : tra.append([node1, node2])  

train = tra    
    
for dt in range(0,len(test)) :
    node1 = test[dt][0] if test[dt][0] <= test[dt][1] else test[dt][1]  
    node2 = test[dt][0] if test[dt][0] >= test[dt][1] else test[dt][1]
    if node1 == node2 : continue
    else : 
      tst.append([node1, node2])
      testSet.add(str(node1)+" "+str(node2))
      
test = tst    
 

In [ ]:
graph = nx.Graph()
for i in nodes_input: 
    graph.add_node(i)

for dt in range(0,len(train)):
    graph.add_edge(train[dt][0],train[dt][1])
 

print(len(graph.edges))
print(len(graph.nodes))

2193
198


In [ ]:
 all_pairs_of_nodes = Pair_Of_Nodes(graph)
common_neighbour_dictionary = Common_NeighboursList(graph, all_pairs_of_nodes)  


pairs_To_Be_Examined = set()
for node1, node2 in all_pairs_of_nodes :
    pairs_To_Be_Examined.add((node1, node2))
    independent_variable[str(node1)+" "+str(node2)] = []
    dependent_variable[str(node1)+" "+str(node2)] = 0
     
print(len(pairs_To_Be_Examined)) # val1
# removing already present edges
for dt in range(0,len(train)):
    node1 = train[dt][0] if train[dt][0] <= train[dt][1] else train[dt][1]  
    node2 = train[dt][0] if train[dt][0] >= train[dt][1] else train[dt][1]
    pairs_To_Be_Examined.discard((node1, node2))
    dependent_variable[str(node1)+" "+str(node2)] = 1
    
for dt in range(0,len(test)):
    node1 = test[dt][0] if test[dt][0] <= test[dt][1] else test[dt][1]  
    node2 = test[dt][0] if test[dt][0] >= test[dt][1] else test[dt][1]
    dependent_variable[str(node1)+" "+str(node2)] = 1  
print(len(pairs_To_Be_Examined))

19503
17310


In [ ]:
common_Neighbours = Common_NeighboursList(graph,all_pairs_of_nodes)

# comparing efficiencies    
#centrality_measures = [closeness_Centrality]
similarity_measures = [common_Neighbours]

similarity_measures_name = ["common_Neighbours"]

for node1, node2 in all_pairs_of_nodes :
    sim_arr = []
    for j in range(len(similarity_measures_name)):
      sim_arr.append(similarity_measures[j][str(node1)+" "+str(node2)]) 
    independent_variable[str(node1)+" "+str(node2)] = sim_arr


In [ ]:
# this feature table contains the Common neighbours and Closeness Centrality node pair wise
feature_table = []
for node1, node2 in pairs_To_Be_Examined:
    close_centrality = Closeness_Centrality(node1,node2, graph)
    temp = [node1,node2,int(common_Neighbours[str(node1)+" "+str(node2)]),close_centrality]
    feature_table.append(temp)

In [ ]:
#here to check the predictions for a particular alpha
y_compare = []
results = []
nodes_value = []
control_values = [0.6]
for x in control_values:
    alpha = x
    yActual = []
    yPredicted = []  
    for row in feature_table :
        node1 = row[0]
        node2 = row[1]
        value = alpha*row[2] + ( (1-alpha)*row[3] )
        yPredicted.append(value)
        nodePair = str(node1)+" "+str(node2)
        if nodePair in testSet: yActual.append(int(1))
        else : yActual.append(int(0))
        nodes_value.append([node1, node2, value])
    auc_roc = roc_auc_score(yActual, yPredicted)
    pre, re, thres = precision_recall_curve(yActual,yPredicted)
    auc_prc = auc(re, pre)
    accuracyParameters = []
    accuracyParameters.append(alpha)
    accuracyParameters.append(auc_roc)
    accuracyParameters.append(auc_prc)
    results.append(accuracyParameters)

#sorting in decreasing order of the value
nodes_value = sorted(nodes_value,key=lambda x: (x[2]), reverse=True)
max_val = nodes_value[0][2]

# value which is more than 50 percent is being predicted as a link to be
for row in nodes_value:
    node1 = row[0]
    node2 = row[1]
    value = row[2]
    nodePair = str(node1)+" "+str(node2)
    percent = (value/max_val)*100
    if percent >= 50:
      row.append(1)
    else : row.append(0)
    if nodePair in testSet:
      row.append(1)
    else : row.append(0)
    
predictions = pd.DataFrame(nodes_value)

In [ ]:
#here i am using various alpha values to see the effect on auc roc values
results = []
nodes_value = []
control_values = np.arange(0,1.05,0.05)
print(control_values)
for x in control_values:
    alpha = x
    yActual = []
    yPredicted = []
    for row in feature_table :
        node1 = row[0]
        node2 = row[1]
        value = alpha*row[2] + ( (1-alpha)*row[3] )
        yPredicted.append(value)
        nodePair = str(node1)+" "+str(node2)
        if nodePair in testSet : yActual.append(int(1))
        else : yActual.append(int(0))
        nodes_value.append([value, node1, node2])
    auc_roc = roc_auc_score(yActual, yPredicted)
    pre, re, thres = precision_recall_curve(yActual,yPredicted)
    auc_prc = auc(re, pre)
    accuracyParameters = []
    accuracyParameters.append(alpha)
    accuracyParameters.append(auc_roc)
    accuracyParameters.append(auc_prc)
    results.append(accuracyParameters)

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


In [ ]:
#stats = pd.DataFrame(results,columns = ["centrality","similarity","auc-roc","auc-prc"])
stats = pd.DataFrame(results,columns = ["alpha","auc-roc","auc-prc"])
print(stats)
stats.to_csv('jazz-result.csv') 

    alpha   auc-roc   auc-prc
0    0.00  0.718740  0.521623
1    0.05  0.940623  0.591431
2    0.10  0.940623  0.591431
3    0.15  0.940623  0.591431
4    0.20  0.940623  0.591431
5    0.25  0.940623  0.591431
6    0.30  0.940623  0.591431
7    0.35  0.940623  0.591431
8    0.40  0.940623  0.591431
9    0.45  0.940623  0.591431
10   0.50  0.940623  0.591431
11   0.55  0.940623  0.591431
12   0.60  0.940623  0.591431
13   0.65  0.940623  0.591431
14   0.70  0.940623  0.591431
15   0.75  0.940623  0.591431
16   0.80  0.940623  0.591431
17   0.85  0.940623  0.591431
18   0.90  0.940623  0.591431
19   0.95  0.940623  0.591431
20   1.00  0.941359  0.591481


In [ ]:
files.download('jazz-result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>